# The Cell Magics in `IPython`

(Hands on; 15 min)

R. Burke Squires

https://github.com/burkesquires

IPython has a system of commands we call 'magics' that provide effectively a mini command language that is orthogonal to the syntax of Python and is extensible by the user with new commands.  Magics are meant to be typed interactively, so they use command-line conventions, such as using whitespace for separating arguments, dashes for options and other conventions typical of a command-line environment.

Magics come in two kinds:

* Line magics: these are commands prepended by one `%` character and whose arguments only extend to the end of the current line.
* Cell magics: these use *two* percent characters as a marker (`%%`), and they receive as argument *both* the current line where they are declared and the whole body of the cell.  Note that cell magics can *only* be used as the first line in a cell, and as a general principle they can't be 'stacked' (i.e. you can only use one cell magic per cell).  A few of them, because of how they operate, can be stacked, but that is something you will discover on a case by case basis.

The `%lsmagic` magic is used to list all available magics, and it will show both line and cell magics currently defined:

In [20]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

Since in the introductory section we already covered the most frequently used line magics, we will focus here on the cell magics, which offer a great amount of power.

Let's load matplotlib and numpy so we can use numerics/plotting at will later on.

    %matplotlib inline
    import numpy as np
    import matplotlib.pyplot as plt

## <!--====--> Some simple cell magics

Timing the execution of code; the 'timeit' magic exists both in line and cell form:

    %timeit np.linalg.eigvals(np.random.rand(100,100))

    %%timeit a = np.random.rand(100, 100)
    np.linalg.eigvals(a)

The `%%capture` magic can be used to capture the stdout/err of any block of python code, either to discard it (if it's noise to you) or to store it in a variable for later use:

In [1]:
    %%capture capt
    from __future__ import print_function
    import sys
    print('Hello stdout')
    print('and stderr', file=sys.stderr)

In [2]:
    capt.stdout, capt.stderr

('Hello stdout\n', 'and stderr\n')

    capt.show()

The `%%writefile` magic is a very useful tool that writes the cell contents as a named file:

    %%writefile foo.py
    print('Hello world')

    %run foo

## <!--====--> Magics for running code under other interpreters

IPython has a `%%script` cell magic, which lets you run a cell in
a subprocess of any interpreter on your system, such as: bash, ruby, perl, zsh, R, etc.

It can even be a script of your own, which expects input on stdin.

To use it, simply pass a path or shell command to the program you want to run on the `%%script` line,
and the rest of the cell will be run by that script, and stdout/err from the subprocess are captured and displayed.

    %%script python
    import sys
    print 'hello from Python %s' % sys.version

    %%script python3
    import sys
    print('hello from Python: %s' % sys.version)

IPython also creates aliases for a few common interpreters, such as bash, ruby, perl, etc.

These are all equivalent to `%%script <name>`

    %%ruby
    puts "Hello from Ruby #{RUBY_VERSION}"

    %%bash
    echo "hello from $BASH"

## Exercise: write your own script that numbers input lines

Write a file, called `lnum.py`, such that the following cell works as shown (hint: don't forget about the executable bit!): 

    %%script ./lnum.py
    my first line
    my second
    more

## Capturing output

You can also capture stdout/err from these subprocesses into Python variables, instead of letting them go directly to stdout/err

    %%bash
    echo "hi, stdout"
    echo "hello, stderr" >&2

    %%bash --out output --err error
    echo "hi, stdout"
    echo "hello, stderr" >&2

    print(error)
    print(output)

## Background Scripts

These scripts can be run in the background, by adding the `--bg` flag.

When you do this, output is discarded unless you use the `--out/err`
flags to store output as above.

    %%ruby --bg --out ruby_lines
    for n in 1...10
        sleep 1
        puts "line #{n}"
        STDOUT.flush
    end

When you do store output of a background thread, these are the stdout/err *pipes*,
rather than the text of the output.

    ruby_lines

    print(ruby_lines.read())